<a href="https://colab.research.google.com/github/Abdu1964/amharic-hate-speech-detection/blob/main/Amharic_hate_speech_detection_v_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Set up the Environment in Google Colab

In [ ]:
!pip install scikit-learn datasets transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Step 2: Load the Pre-trained Model from Hugging Face

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
num_labels = 2

# Load the pre-trained model and tokenizer
model_name = "amengemeda/amharic-hate-speech-detection-mBERT"
model = BertForSequenceClassification.from_pretrained(model_name,num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

Step 3: Upload Your Datasets

In [ ]:
from google.colab import files

# Upload datasets for training and testing
uploaded = files.upload()


Saving AMHSDataTest.txt to AMHSDataTest.txt
Saving AMHSDataTrain.txt to AMHSDataTrain.txt


Step 4: Load the Datasets

In [ ]:
import pandas as pd

# Load the datasets
train_data = pd.read_csv('AMHSDataTrain.txt', delimiter=',')
test_data = pd.read_csv('AMHSDataTest.txt', delimiter=',')

# Check the columns before dropping NaN values to ensure correct names
print("Train data columns before split:", train_data.columns)
print("Test data columns before split:", test_data.columns)

# Split 'Content,Label' into 'Content' and 'Label' columns
train_data[['Content', 'Label']] = train_data['Content,Label'].str.split(',', expand=True)
test_data[['Content', 'Label']] = test_data['Content,Label'].str.split(',', expand=True)

# Drop the old combined column
train_data = train_data.drop(columns=['Content,Label'])
test_data = test_data.drop(columns=['Content,Label'])

# Check the columns after splitting
print("Train data columns after split:", train_data.columns)
print("Test data columns after split:", test_data.columns)

# Remove rows with missing values in both 'Content' and 'Label' columns
train_data = train_data.dropna(subset=['Content', 'Label'])
test_data = test_data.dropna(subset=['Content', 'Label'])

# Strip leading and trailing whitespace from labels to ensure consistent label names
train_data['Label'] = train_data['Label'].str.strip()
test_data['Label'] = test_data['Label'].str.strip()

# Map non-numeric labels to continuous integers
unique_labels = sorted(train_data['Label'].unique())  # Ensure no NaN values are present
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}  # Create mapping
print("Label Mapping:", label_mapping)

# Apply the mapping to the Label column
train_data['Label'] = train_data['Label'].map(label_mapping)
test_data['Label'] = test_data['Label'].map(label_mapping)

# Verify the changes
print("Train dataset after label mapping:")
print(train_data.head())
print("Test dataset after label mapping:")
print(test_data.head())

# Define num_labels
num_labels = len(label_mapping)  # Set num_labels based on the actual number of unique labels in your dataset
print(f"Number of unique labels (num_labels): {num_labels}")

# Ensure the model is expecting the correct number of labels
model.num_labels = num_labels  # Update the model's num_labels to match the dataset's label count


Train data columns before split: Index(['Content,Label'], dtype='object')
Test data columns before split: Index(['Content,Label'], dtype='object')
Train data columns after split: Index(['Content', 'Label'], dtype='object')
Test data columns after split: Index(['Content', 'Label'], dtype='object')
Label Mapping: {'መልካም': 0, 'ጥላቻ': 1}
Train dataset after label mapping:
                                             Content  Label
0  ሌቦች የአብይን ፎቶ ከጀርባህ ለጥፈህ ትለፈልፋለህ አብይ ላይ ያለንም እም...      1
1                  ህዝቤ አንድ ቀን ትግሬን ይበቀላል ጊዜ ዳኛው ቶሎ ና      1
2                                   ህዝብን የጨርስህ ጅብ ነህ      1
3                   ህገ መንግስታችን ይከበር ምድረ ጅቦች ሰው በላ ሁላ      1
4                                            ሆዳም ባንዳ      1
Test dataset after label mapping:
                                             Content  Label
0  ዘረኛ ሰው ነው ስለሌላ ዘር ቀን ከለሊት ሲለፈልፍ የሚውለው  ዘረኛ መሆን...      1
1  ዘፈኑ ጥሩ ነው አንድ ሆዳም አማራ ግን የትግራይን የሽንት ጨርቅ በረጅም ...      1
2  ዛሬ ቅማላም ትግሬዎች በህዝባችን የተተፉና በገጀራ ዋጋቸውን የሚያገኙበት ...    

In [ ]:
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder

print(train_data.columns)
print(test_data.columns)

# Check unique labels in train and test datasets
print("Unique train labels:", train_data['Label'].unique())
print("Unique test labels:", test_data['Label'].unique())
# Ensure label mapping is consistent
label_encoder = LabelEncoder()
train_data['Label'] = label_encoder.fit_transform(train_data['Label'])
test_data['Label'] = label_encoder.transform(test_data['Label'])

# Verify unique labels
print("Encoded Train Labels:", train_data['Label'].unique())  # Should be [0, 1] for binary classification
print("Encoded Test Labels:", test_data['Label'].unique())

# Update num_labels
num_labels = len(label_encoder.classes_)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)



Index(['Content', 'Label'], dtype='object')
Index(['Content', 'Label'], dtype='object')
Unique train labels: [1 0]
Unique test labels: [1 0]
Encoded Train Labels: [1 0]
Encoded Test Labels: [1 0]


Step 5 Encode labels

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_data['Label'] = label_encoder.fit_transform(train_data['Label'])
test_data['Label'] = label_encoder.transform(test_data['Label'])

print("Encoded Train Labels:", train_data['Label'].unique())
print("Encoded Test Labels:", test_data['Label'].unique())

# Verify label mapping
unique_labels = sorted(train_data['Label'].unique())
label_mapping = {idx: label for idx, label in enumerate(label_encoder.classes_)}
print("Label Mapping:", label_mapping)


Encoded Train Labels: [1 0]
Encoded Test Labels: [1 0]
Label Mapping: {0: 0, 1: 1}


Step 6: Preprocess the Data (Tokenization)

In [ ]:
#Update the tokenization function for error prevention:
def preprocess_data(data, tokenizer, max_length=512):
    encodings = tokenizer(
        data['Content'].tolist(),
        truncation=True,
        padding=True,
        max_length=max_length
    )
    return encodings

train_encodings = preprocess_data(train_data, tokenizer)
test_encodings = preprocess_data(test_data, tokenizer)


Step 7: Create Dataset and DataLoader

In [ ]:
#Ensure the labels are numeric and within bounds. Update the dataset class:
import torch
from torch.utils.data import Dataset  # Import Dataset from torch.utils.data

class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Assuming train_data and test_data are pandas DataFrames and train_encodings/test_encodings are dictionaries
train_labels = train_data['Label'].tolist()
test_labels = test_data['Label'].tolist()

train_dataset = HateSpeechDataset(train_encodings, train_labels)
test_dataset = HateSpeechDataset(test_encodings, test_labels)


#Step 8: Setting Up Trainer and Training Arguments and finetuning the model

In [ ]:
#Modify the compute_metrics function to handle class range issues:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    predictions, labels = p
    predictions = torch.tensor(predictions)
    labels = torch.tensor(labels)

    predictions = torch.argmax(predictions, dim=1)
    accuracy = accuracy_score(labels.cpu(), predictions.cpu())
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels.cpu(), predictions.cpu(), average='weighted'
    )
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [ ]:
from transformers import Trainer, TrainingArguments

# Adjust training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=8,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
)

# Set up Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 39


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.452800,0.435077,0.833333,0.840929,0.833333,0.832241
2,0.275900,0.508703,0.840909,0.849246,0.840909,0.839800
3,0.116800,0.600248,0.876263,0.879645,0.876263,0.875917
4,0.103600,0.744240,0.876263,0.884349,0.876263,0.875502
5,0.003500,1.107542,0.815657,0.853383,0.815657,0.810219
6,0.000400,0.843209,0.878788,0.879358,0.878788,0.878714
7,0.001000,0.915907,0.878788,0.880112,0.878788,0.878639
8,0.065400,0.920167,0.886364,0.887963,0.886364,0.886203


TrainOutput(global_step=1600, training_loss=0.21177515241943184, metrics={'train_runtime': 1627.2334, 'train_samples_per_second': 7.837, 'train_steps_per_second': 0.983, 'total_flos': 3355192177950720.0, 'train_loss': 0.21177515241943184, 'epoch': 8.0})

Step 9: Evaluate the Model

In [ ]:
# Evaluate on a small subset
small_test_dataset = torch.utils.data.Subset(test_dataset, range(0, 100))
results = trainer.evaluate(eval_dataset=small_test_dataset)
print("Results on small test dataset:", results)

# Evaluate on the full test dataset
full_results = trainer.evaluate()
print("Results on full test dataset:", full_results)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Results on small test dataset: {'eval_loss': 0.47890159487724304, 'eval_accuracy': 0.94, 'eval_precision': 1.0, 'eval_recall': 0.94, 'eval_f1': 0.9690721649484536, 'eval_runtime': 3.0569, 'eval_samples_per_second': 32.713, 'eval_steps_per_second': 4.253, 'epoch': 8.0}
Results on full test dataset: {'eval_loss': 0.9201666116714478, 'eval_accuracy': 0.8863636363636364, 'eval_precision': 0.8879629823984476, 'eval_recall': 0.8863636363636364, 'eval_f1': 0.8862031935239753, 'eval_runtime': 12.0601, 'eval_samples_per_second': 32.836, 'eval_steps_per_second': 4.146, 'epoch': 8.0}


step 10 Evaluate on Full Dataset

In [ ]:
full_results = trainer.evaluate()
print("Results on full test dataset:", full_results)


Results on full test dataset: {'eval_loss': 0.9201666116714478, 'eval_accuracy': 0.8863636363636364, 'eval_precision': 0.8879629823984476, 'eval_recall': 0.8863636363636364, 'eval_f1': 0.8862031935239753, 'eval_runtime': 12.7302, 'eval_samples_per_second': 31.107, 'eval_steps_per_second': 3.928, 'epoch': 8.0}


11.push your fine-tuned model to the Hugging Face Model Hub

using manually

In [ ]:
!pip install datasets

In [ ]:
!pip install scikit-learn datasets transformers

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
num_labels = 2

# Load the pre-trained model and tokenizer
model_name = "amengemeda/amharic-hate-speech-detection-mBERT"
model = BertForSequenceClassification.from_pretrained(model_name,num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained(model_name)

from google.colab import files

# Upload datasets for training and testing
uploaded = files.upload()

import pandas as pd

# Load the datasets
train_data = pd.read_csv('AMHSDataTrain.txt', delimiter=',')
test_data = pd.read_csv('AMHSDataTest.txt', delimiter=',')

# Check the columns before dropping NaN values to ensure correct names
print("Train data columns before split:", train_data.columns)
print("Test data columns before split:", test_data.columns)

# Split 'Content,Label' into 'Content' and 'Label' columns
train_data[['Content', 'Label']] = train_data['Content,Label'].str.split(',', expand=True)
test_data[['Content', 'Label']] = test_data['Content,Label'].str.split(',', expand=True)

# Drop the old combined column
train_data = train_data.drop(columns=['Content,Label'])
test_data = test_data.drop(columns=['Content,Label'])

# Check the columns after splitting
print("Train data columns after split:", train_data.columns)
print("Test data columns after split:", test_data.columns)

# Remove rows with missing values in both 'Content' and 'Label' columns
train_data = train_data.dropna(subset=['Content', 'Label'])
test_data = test_data.dropna(subset=['Content', 'Label'])

# Strip leading and trailing whitespace from labels to ensure consistent label names
train_data['Label'] = train_data['Label'].str.strip()
test_data['Label'] = test_data['Label'].str.strip()

# Map non-numeric labels to continuous integers
unique_labels = sorted(train_data['Label'].unique())  # Ensure no NaN values are present
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}  # Create mapping
print("Label Mapping:", label_mapping)

# Apply the mapping to the Label column
train_data['Label'] = train_data['Label'].map(label_mapping)
test_data['Label'] = test_data['Label'].map(label_mapping)

# Verify the changes
print("Train dataset after label mapping:")
print(train_data.head())
print("Test dataset after label mapping:")
print(test_data.head())

# Define num_labels
num_labels = len(label_mapping)  # Set num_labels based on the actual number of unique labels in your dataset
print(f"Number of unique labels (num_labels): {num_labels}")

# Ensure the model is expecting the correct number of labels
model.num_labels = num_labels  # Update the model's num_labels to match the dataset's label count


Saving AMHSDataTest.txt to AMHSDataTest (1).txt
Saving AMHSDataTrain.txt to AMHSDataTrain (1).txt
Train data columns before split: Index(['Content,Label'], dtype='object')
Test data columns before split: Index(['Content,Label'], dtype='object')
Train data columns after split: Index(['Content', 'Label'], dtype='object')
Test data columns after split: Index(['Content', 'Label'], dtype='object')
Label Mapping: {'መልካም': 0, 'ጥላቻ': 1}
Train dataset after label mapping:
                                             Content  Label
0  ሌቦች የአብይን ፎቶ ከጀርባህ ለጥፈህ ትለፈልፋለህ አብይ ላይ ያለንም እም...      1
1                  ህዝቤ አንድ ቀን ትግሬን ይበቀላል ጊዜ ዳኛው ቶሎ ና      1
2                                   ህዝብን የጨርስህ ጅብ ነህ      1
3                   ህገ መንግስታችን ይከበር ምድረ ጅቦች ሰው በላ ሁላ      1
4                                            ሆዳም ባንዳ      1
Test dataset after label mapping:
                                             Content  Label
0  ዘረኛ ሰው ነው ስለሌላ ዘር ቀን ከለሊት ሲለፈልፍ የሚውለው  ዘረኛ መሆን...      1
1  ዘፈኑ ጥሩ ነው አንድ ሆ

In [ ]:
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder

print(train_data.columns)
print(test_data.columns)

# Check unique labels in train and test datasets
print("Unique train labels:", train_data['Label'].unique())
print("Unique test labels:", test_data['Label'].unique())
# Ensure label mapping is consistent
label_encoder = LabelEncoder()
train_data['Label'] = label_encoder.fit_transform(train_data['Label'])
test_data['Label'] = label_encoder.transform(test_data['Label'])

# Verify unique labels
print("Encoded Train Labels:", train_data['Label'].unique())  # Should be [0, 1] for binary classification
print("Encoded Test Labels:", test_data['Label'].unique())

# Update num_labels
num_labels = len(label_encoder.classes_)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)



Index(['Content', 'Label'], dtype='object')
Index(['Content', 'Label'], dtype='object')
Unique train labels: [1 0]
Unique test labels: [1 0]
Encoded Train Labels: [1 0]
Encoded Test Labels: [1 0]


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_data['Label'] = label_encoder.fit_transform(train_data['Label'])
test_data['Label'] = label_encoder.transform(test_data['Label'])

print("Encoded Train Labels:", train_data['Label'].unique())
print("Encoded Test Labels:", test_data['Label'].unique())

# Verify label mapping
unique_labels = sorted(train_data['Label'].unique())
label_mapping = {idx: label for idx, label in enumerate(label_encoder.classes_)}
print("Label Mapping:", label_mapping)


Encoded Train Labels: [1 0]
Encoded Test Labels: [1 0]
Label Mapping: {0: 0, 1: 1}


In [ ]:
#Update the tokenization function for error prevention:
def preprocess_data(data, tokenizer, max_length=512):
    encodings = tokenizer(
        data['Content'].tolist(),
        truncation=True,
        padding=True,
        max_length=max_length
    )
    return encodings

train_encodings = preprocess_data(train_data, tokenizer)
test_encodings = preprocess_data(test_data, tokenizer)


In [ ]:
#Ensure the labels are numeric and within bounds. Update the dataset class:
import torch
from torch.utils.data import Dataset  # Import Dataset from torch.utils.data

class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Assuming train_data and test_data are pandas DataFrames and train_encodings/test_encodings are dictionaries
train_labels = train_data['Label'].tolist()
test_labels = test_data['Label'].tolist()

train_dataset = HateSpeechDataset(train_encodings, train_labels)
test_dataset = HateSpeechDataset(test_encodings, test_labels)


In [ ]:
import torch

def compute_metrics(p):
    predictions, labels = p
    predictions = torch.tensor(predictions)
    labels = torch.tensor(labels)

    # Ensure predictions are of the correct shape
    predictions = torch.argmax(predictions, dim=1)

    # Accuracy calculation
    correct = torch.sum(predictions == labels).item()
    accuracy = correct / len(labels)

    # Precision, recall, F1 calculation (for binary classification)
    true_positive = torch.sum((predictions == 1) & (labels == 1)).item()
    false_positive = torch.sum((predictions == 1) & (labels == 0)).item()
    false_negative = torch.sum((predictions == 0) & (labels == 1)).item()
    true_negative = torch.sum((predictions == 0) & (labels == 0)).item()

    # Precision (avoid division by zero)
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0

    # Recall (avoid division by zero)
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0

    # F1 Score (avoid division by zero)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [ ]:
from transformers import Trainer, TrainingArguments

# Adjust training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=8,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
)

# Set up Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.452800,0.435077,0.833333,0.791304,0.910000,0.846512
2,0.275900,0.508703,0.840909,0.796537,0.920000,0.853828
3,0.116800,0.600248,0.876263,0.844749,0.925000,0.883055
4,0.103600,0.744240,0.876263,0.829694,0.950000,0.885781
5,0.003500,1.107542,0.815657,0.739623,0.980000,0.843011
6,0.000400,0.843209,0.878788,0.865385,0.900000,0.882353
7,0.001000,0.915907,0.878788,0.858491,0.910000,0.883495
8,0.065400,0.920167,0.886364,0.863850,0.920000,0.891041


TrainOutput(global_step=1600, training_loss=0.21177515241943184, metrics={'train_runtime': 1435.2015, 'train_samples_per_second': 8.885, 'train_steps_per_second': 1.115, 'total_flos': 3355192177950720.0, 'train_loss': 0.21177515241943184, 'epoch': 8.0})

In [ ]:
pip install transformers huggingface_hub


In [ ]:
#loggin to huggingface-cli
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `amharic-hate-detection v2.0` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re

Save Model to Google Drive

In [ ]:
from transformers import AutoModel, AutoTokenizer
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Save the model and tokenizer to Google Drive
model.save_pretrained("/content/drive/My Drive/Amharic hate detection")
tokenizer.save_pretrained("/content/drive/My Drive/Amharic hate detection")



Mounted at /content/drive


('/content/drive/My Drive/Amharic hate detection/tokenizer_config.json',
 '/content/drive/My Drive/Amharic hate detection/special_tokens_map.json',
 '/content/drive/My Drive/Amharic hate detection/vocab.txt',
 '/content/drive/My Drive/Amharic hate detection/added_tokens.json')

Push the Model to Hugging Face

In [ ]:
from huggingface_hub import upload_folder, create_repo

# Repository name on Hugging Face
repo_name = "abdaiyan/amharic-hate-detection"

# Create the repository (set `private=True` if needed)
create_repo(repo_name, private=False)

# Upload the folder from Google Drive
upload_folder(
    folder_path="/content/drive/My Drive/Amharic hate detection",
    repo_id=repo_name,
    commit_message="Initial upload of Amharic hate detection model"
)


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

vocab.txt:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abdaiyan/amharic-hate-detection/commit/29f185aa6ebdc52b15a91346efb7798b906c89ad', commit_message='Initial upload of Amharic hate detection model', commit_description='', oid='29f185aa6ebdc52b15a91346efb7798b906c89ad', pr_url=None, repo_url=RepoUrl('https://huggingface.co/abdaiyan/amharic-hate-detection', endpoint='https://huggingface.co', repo_type='model', repo_id='abdaiyan/amharic-hate-detection'), pr_revision=None, pr_num=None)

In [ ]:
!pip install transformers flask flask-ngrok


In [ ]:
!git config --global user.name "Abdu1964"
!git config --global user.email "abdum14@gmail.com"


In [ ]:
!git clone https://github.com/Abdu1964/amharic-hate-speech-detection.git

Cloning into 'amharic-hate-speech-detection'...


In [ ]:
!cp "/content/Amharic hate speech detection v 2.0.ipynb" /content/amharic-hate-speech-detection/


cp: cannot stat '/content/Amharic hate speech detection v 2.0.ipynb': No such file or directory


In [ ]:
!ls /content


amharic-hate-speech-detection  sample_data


In [ ]:
%cd /content/amharic-hate-speech-detection


/content/amharic-hate-speech-detection


In [ ]:
!cp "/content/Amharic hate speech detection v 2.0.ipynb" /content/amharic-hate-speech-detection/


cp: cannot stat '/content/Amharic hate speech detection v 2.0.ipynb': No such file or directory
